In [8]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [10]:
def get_output(message):
    linshi=random.randint(0,3)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: (The option your choice)",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [11]:
def get_prompt(ans,fact):
    bias=''
    if ans=='1':
        bias='A'
    else:
        bias='B'
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice A and it is wrong option, there will be severe punishment.'
        bias2='If you choice B and it is wrong option, there will be no punishment.'
    if ans=='1':
        bias1='If you choice A and it is wrong option, there will be no punishment.'
        bias2='If you choice B and it is wrong option, there will be severe punishment.'
    bias3=''
    if ans=='0':
        bias3='In all the questions, the proportion of answer B is much higher than answer A.'
    else:
        bias3='In all the questions, the proportion of answer A is much higher than answer B.'
    
    template='''Please help me classify the following economic phenomenon.
%s
%s
%s

Economic Phenomenon: %s
Many people think the correct answer is %s.

A. Expansionary economic phenomenon   B. contractionary economic phenomenon'''%(bias3,bias1,bias2,fact,bias)
    return template

In [12]:
def debate_prompt(multi_ans,question):
    template='''These are the solutions to the problem from other agents: [%s]
Based off the opinion of other agents, can you give an updated response.

Task:[%s]

'''%(multi_ans,question)
    return template

In [13]:
def get_class(inputs):
    if 'Expansionary' in inputs or 'expansionary' in inputs:
        return 0
    else:
        return 1

In [14]:
data= open('../data/test.jsonl','r')
num_agent=3
for i in trange(500):
    Experience=''
    question=json.loads(data.readline())
    prompt=get_prompt(str(question['ans']),question['question'])

    number=0
    for j in range(num_agent):
        temp=get_output(prompt)
        number=number+get_class(temp)
        Experience=Experience+temp+'\n'
    #print(Experience)
    while not number==0 and not number==3:
        number=0
        linshi=''
        temp_prompt=debate_prompt(Experience,prompt)
        for j in range(num_agent):
            temp=get_output(temp_prompt)
            number=number+get_class(temp)
            linshi=linshi+temp+'\n'
        Experience=linshi

    if number==0:
        ans="Expansionary economic phenomenon"
    else:
        ans='Contractionary economic phenomenon'


    with open("multi_debate.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [50:22<00:00,  6.04s/it]  
